# Regressão logística

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

Este capítulo introduz dois tópicos relacionados: as probabilidades logísticas e a regressão logística.

Em <<_BayessRule>>, reescrevemos o Teorema de Bayes em termos de probabilidades e a Regra de Bayes derivada, o que pode ser uma forma conveniente de fazer uma actualização Bayesiana em papel ou na sua cabeça.

Neste capítulo, analisaremos a Regra de Bayes numa escala logarítmica, que fornece uma visão sobre como acumulamos provas através de actualizações sucessivas.

Isto leva directamente à regressão logística, que se baseia num modelo linear da relação entre a evidência e as probabilidades logísticas de uma hipótese.

Como exemplo, utilizaremos dados do Vaivém Espacial para explorar a relação entre a temperatura e a probabilidade de danos nos O-rings.

Como exercício, terá a oportunidade de modelar a relação entre a idade da criança quando inicia a escola e a sua probabilidade de ser diagnosticada com Distúrbio de Défice de Atenção e Hiperactividade (DDAH).

## Probabilidades de registo

Quando estava na faculdade, inscrevi-me para uma aula sobre a Teoria da Computação.

No primeiro dia de aula, fui o primeiro a chegar.

Uns minutos depois, chegou outro estudante.

Na altura, cerca de 83% dos estudantes do programa de ciências informáticas [were male](https://www.aps.org/programs/education/statistics/fraction-phd.cfm), por isso fiquei ligeiramente surpreendido ao notar que o outro estudante era do sexo feminino.

Quando outra aluna chegou alguns minutos mais tarde, comecei a pensar que estava na sala errada.

Quando a terceira aluna chegou, eu estava confiante de que estava na sala errada.

E como acabou por acontecer, eu estava.

Vou usar esta anedota para demonstrar a Regra de Bayes numa escala logarítmica e mostrar como ela se relaciona com a regressão logística.

Usando $H$ para representar a hipótese de que eu estava na sala certa, e $F$ para representar a observação de que o primeiro outro estudante era do sexo feminino, podemos escrever a Regra de Bayes desta forma:

$$O(H|F) = O(H) \frac{P(F|H)}{P(F|não H)}$$$

Antes de ver os outros estudantes, estava confiante de que estava na sala certa, pelo que poderia atribuir uma probabilidade prévia de 10:1 a favor:

$$O(H) = 10$$$



Se eu estivesse na sala certa, a probabilidade da primeira estudante do sexo feminino era de cerca de 17%.

Se eu não estivesse no quarto certo, a probabilidade da primeira aluna era mais de 50%, 

$$$frac{P(F|H)}{P(F|não H)} = 17 / 50$$$

Portanto, o rácio de probabilidade está próximo de 1/3.  Aplicando a regra de Bayes, as probabilidades posteriores eram

$$O(H|F) = 10 / 3$$$

Depois de dois estudantes, as probabilidades posteriores eram

$$O(H|FF) = 10 / 9$$$

E depois de três estudantes:

$$O(H|FFF) = 10 / 27$$$

Nessa altura, estava certo ao suspeitar que estava na sala errada.

A tabela seguinte mostra as probabilidades após cada actualização, as probabilidades correspondentes, e a mudança na probabilidade após cada passo, expressa em pontos percentuais.

In [4]:
def prob(o):
    return o / (o+1)

In [5]:
import pandas as pd

index = ['prior', '1 student', '2 students', '3 students']

table = pd.DataFrame(index=index)
table['odds'] = [10, 10/3, 10/9, 10/27]
table['prob'] = prob(table['odds'])
table['prob diff'] = table['prob'].diff() * 100
table.fillna('--')

Cada actualização utiliza a mesma probabilidade, mas as alterações na probabilidade não são as mesmas.  A primeira actualização diminui a probabilidade em cerca de 14 pontos percentuais, a segunda em 24, e a terceira em 26.

Isso é normal para este tipo de actualização, e de facto é necessário; se as alterações fossem do mesmo tamanho, entraríamos rapidamente em probabilidades negativas.

As probabilidades seguem um padrão mais óbvio.  Porque cada actualização multiplica as probabilidades pelo mesmo rácio de probabilidade, as probabilidades formam uma sequência geométrica.

E isso leva-nos a considerar outra forma de representar a incerteza: **d probabilidades do logaritmo**, que é o logaritmo das probabilidades, geralmente expresso usando o log natural (base $e$).

Acrescentar probabilidades de registo à tabela:

In [6]:
import numpy as np

table['log odds'] = np.log(table['odds'])
table['log odds diff'] = table['log odds'].diff()
table.fillna('--')

Poderá reparar:

* Quando a probabilidade é superior a 0,5, as probabilidades são superiores a 1, e as probabilidades de registo são positivas.

* Quando a probabilidade é inferior a 0,5, as probabilidades são inferiores a 1, e as probabilidades de registo são negativas.

Também se pode notar que as probabilidades de registo são igualmente espaçadas.

A alteração nas probabilidades de registo após cada actualização é o logaritmo do rácio de probabilidade.

In [7]:
np.log(1/3)

Isso é verdade neste exemplo, e podemos mostrar que é verdade em geral, tomando o diário de bordo de ambos os lados do Bayes's Rule.

$$\log O(H|F) = {P(F|H)}{P(F|H)}{P(F|não H)}$$$

Numa escala de probabilidades de registo, uma actualização Bayesiana é aditiva.  Portanto, se $F^x$ significa que $x$ estudantes do sexo feminino chegam enquanto eu espero, as probabilidades posteriores de log que eu estou na sala certa são:

$$\log O(H|F^x) = \log O(H) + x \log {P(F|H)}{P(F|não H)}$$$

Esta equação representa uma relação linear entre o rácio de probabilidade logarítmica e as probabilidades logarítmicas posteriores.  

Neste exemplo a equação linear é exacta, mas mesmo quando não é, é comum utilizar uma função linear para modelar a relação entre uma variável explicativa, $x$, e uma variável dependente expressa em log odds, como esta:

$$\log O(H | x) = \beta_0 + \beta_1 x$$

onde $\beta_0$ e $\beta_1$ são parâmetros desconhecidos:

* A intercepção, $\beta_0$, é a probabilidade em log da hipótese quando $x$ é 0.

* A inclinação, $\beta_1$, é o registo do rácio de probabilidade.

Esta equação é a base da regressão logística.

## O problema do vaivém espacial

Como exemplo de regressão logística, vou resolver um problema do livro de Cameron Davidson-Pilon, [*Bayesian Methods for Hackers*](http://nbviewer.jupyter.org/github/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/blob/master/Chapter2_MorePyMC/Ch2_MorePyMC_PyMC2.ipynb).  Escreve ele:

> "Em 28 de Janeiro de 1986, o vigésimo quinto voo do programa de vaivém espacial dos EUA terminou em desastre quando um dos lança-foguetes do Vaivém Challenger explodiu pouco depois do seu lançamento, matando todos os sete membros da tripulação. A comissão presidencial sobre o acidente concluiu que este foi causado pela falha de um anel em O numa junta de campo no propulsor do foguetão, e que esta falha se deveu a um desenho defeituoso que tornou o anel em O inaceitavelmente sensível a uma série de factores, incluindo a temperatura exterior. Dos 24 voos anteriores, estavam disponíveis dados sobre falhas dos anéis em O em 23 (um foi perdido no mar), e estes dados foram discutidos na noite anterior ao lançamento do Challenger, mas infelizmente apenas os dados correspondentes aos 7 voos em que houve um incidente de danos foram considerados importantes e pensou-se que estes não mostravam nenhuma tendência óbvia".

O conjunto de dados é originalmente de [this paper](https://amstat.tandfonline.com/doi/abs/10.1080/01621459.1989.10478858), mas também disponível a partir de [Davidson-Pilon](https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter2_MorePyMC/data/challenger_data.csv).

In [8]:
download('https://raw.githubusercontent.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/Chapter2_MorePyMC/data/challenger_data.csv')

Vou ler os dados e fazer alguma limpeza.

In [9]:
data = pd.read_csv('challenger_data.csv', parse_dates=[0])

# avoiding column names with spaces
data.rename(columns={'Damage Incident': 'Damage'}, inplace=True)

# dropping row 3, in which Damage Incident is NaN,
# and row 24, which is the record for the Challenger
data.drop(labels=[3, 24], inplace=True)

# convert the Damage column to integer
data['Damage'] = data['Damage'].astype(int)

data

Aqui estão as primeiras filas:

In [10]:
data.head()

As colunas são:

*Data': A data de lançamento,

* Temperatura: temperatura exterior em Fahrenheit, e

* "Danos": `1` se houvesse um incidente de danos e `0` de outra forma.

Há 23 lançamentos no conjunto de dados, 7 com incidentes de danos.

In [11]:
len(data), data['Damage'].sum()

A figura seguinte mostra a relação entre os danos e a temperatura.

In [12]:
import matplotlib.pyplot as plt
from utils import decorate

def plot_data(data):
    """Plot damage as a function of temperature.
    
    data: DataFrame
    """
    plt.plot(data['Temperature'], data['Damage'], 'o', 
             label='data', color='C0', alpha=0.4)

    decorate(ylabel="Probability of damage",
         xlabel="Outside temperature (deg F)",
         title="Damage to O-Rings vs Temperature")

In [13]:
plot_data(data)

Quando a temperatura exterior era inferior a 65 graus, havia sempre danos nos anéis em O.  Quando a temperatura estava acima dos 65 graus, normalmente não houve danos.  

Com base neste número, parece plausível que a probabilidade de danos esteja relacionada com a temperatura.  Se assumirmos que esta probabilidade segue um modelo logístico, podemos escrever:

$$\log O(H | x) = \beta_0 + \beta_1 x$$

onde $H$ é a hipótese de que os O-rings serão danificados, $x$ é a temperatura, e $\beta_0$ e $\beta_1$ são os parâmetros que iremos estimar. 

Por razões que explicarei em breve, definirei $x$ a ser deslocado de temperatura por um offset de modo a que a sua média seja 0.

In [14]:
offset = data['Temperature'].mean().round()
data['x'] = data['Temperature'] - offset
offset

E, por uma questão de coerência, criarei uma cópia das colunas "Danos" chamadas "sim".

In [15]:
data['y'] = data['Damage']

Antes de fazer uma actualização Bayesiana, vou utilizar `statsmodels' para executar uma regressão logística convencional (não Bayesiana).

In [16]:
import statsmodels.formula.api as smf

formula = 'y ~ x'
results = smf.logit(formula, data=data).fit(disp=False)
results.params

O "resultado" contém uma "estimativa de ponto" para cada parâmetro, ou seja, um único valor em vez de uma distribuição posterior.

A intercepção é de cerca de -1,2, e a inclinação estimada é de cerca de -0,23.

Para ver o significado destes parâmetros, vou usá-los para calcular as probabilidades para uma gama de temperaturas.

Aqui está o alcance:

In [17]:
inter = results.params['Intercept']
slope = results.params['x']
xs = np.arange(53, 83) - offset

Podemos utilizar a equação de regressão logística para calcular as probabilidades logísticas:

In [18]:
log_odds = inter + slope * xs

E depois converter para probabilidades.

In [19]:
odds = np.exp(log_odds)
ps = odds / (odds + 1)

In [20]:
ps.mean()

A conversão das probabilidades de registo em probabilidades é uma operação suficientemente comum que tem um nome, `expit`, e SciPy fornece uma função que a computa.

In [21]:
from scipy.special import expit

ps = expit(inter + slope * xs)

In [22]:
ps.mean()

Eis como se parece o modelo logístico com estes parâmetros estimados.

In [23]:
plt.plot(xs+offset, ps, label='model', color='C1')

plot_data(data)

A baixas temperaturas, a probabilidade de danos é elevada; a altas temperaturas, desce para perto de 0.

Mas isso é baseado na regressão logística convencional.

Agora vamos fazer a versão Bayesiana.

## Distribuição prévia

Vou utilizar distribuições uniformes para ambos os parâmetros, utilizando as estimativas de pontos da secção anterior para me ajudar a escolher os limites superior e inferior.

In [24]:
from utils import make_uniform

qs = np.linspace(-5, 1, num=101)
prior_inter = make_uniform(qs, 'Intercept')

In [25]:
qs = np.linspace(-0.8, 0.1, num=101)
prior_slope = make_uniform(qs, 'Slope')

Podemos utilizar o `make_joint` para construir a distribuição conjunta prévia.

In [26]:
from utils import make_joint

joint = make_joint(prior_inter, prior_slope)

Os valores de "intercepção" correm através das colunas, os valores de "inclinação" correm através das linhas. 

Para este problema, será conveniente "empilhar" o anterior para que os parâmetros sejam níveis num `MultiIndex`, e colocar o resultado num `Pmf`.

In [27]:
from empiricaldist import Pmf

joint_pmf = Pmf(joint.stack())
joint_pmf.head()

O `joint_pmf` é um `Pmf` com dois níveis no índice, um para cada parâmetro.  Isso facilita a passagem de possíveis pares de parâmetros, como veremos na secção seguinte.

## Probabilidade

Para fazer a actualização, temos de calcular a probabilidade dos dados para cada par de parâmetros possíveis.  

Para facilitar isso, vou agrupar os dados por temperatura, `x`, e contar o número de lançamentos e incidentes de danos a cada temperatura.

In [28]:
grouped = data.groupby('x')['y'].agg(['count', 'sum'])
grouped.head()

O resultado é um "DataFrame" com duas colunas: "contagem" é o número de lançamentos a cada temperatura; "soma" é o número de incidentes de danos.

Para ser consistente com os parâmetros das distribuições binomiais, atribuo-os às variáveis chamadas `ns' e `ks'.

In [29]:
ns = grouped['count']
ks = grouped['sum']

Para calcular a probabilidade dos dados, vamos assumir temporariamente que os parâmetros que acabámos de estimar, "inclinação" e "inverno", estão correctos.

Podemos utilizá-los para calcular a probabilidade de danos a cada temperatura de lançamento, desta forma:

In [30]:
xs = grouped.index
ps = expit(inter + slope * xs)

O `ps` contém a probabilidade de danos para cada temperatura de lançamento, de acordo com o modelo.

Agora, para cada temperatura temos `ns', `ps', e `ks';

podemos utilizar a distribuição binomial para calcular a probabilidade dos dados.

In [31]:
from scipy.stats import binom

likes = binom.pmf(ks, ns, ps)
likes

Cada elemento de "gostos" é a probabilidade de ver "k" incidentes de danos em "n" lançamentos se a probabilidade de danos for "p". 

A probabilidade de todo o conjunto de dados ser o produto deste conjunto de dados.

In [32]:
likes.prod()

É assim que calculamos a probabilidade dos dados para um determinado par de parâmetros.

Agora podemos calcular a probabilidade dos dados para todos os pares possíveis:

In [33]:
likelihood = joint_pmf.copy()
for slope, inter in joint_pmf.index:
    ps = expit(inter + slope * xs)
    likes = binom.pmf(ks, ns, ps)
    likelihood[slope, inter] = likes.prod() 

Para inicializar a "probabilidade", fazemos uma cópia do `joint_pmf`, que é uma forma conveniente de garantir que a "probabilidade" tem o mesmo tipo, índice e tipo de dados que o `joint_pmf`.

O loop iterates através dos parâmetros.  Para cada par possível, utiliza o modelo logístico para calcular `ps`, calcula a probabilidade dos dados, e atribui o resultado a uma linha em `risco`.

## A Actualização

Agora podemos calcular a distribuição posterior da forma habitual.

In [34]:
posterior_pmf = joint_pmf * likelihood
posterior_pmf.normalize()

Como utilizámos um anterior uniforme, o par de parâmetros com a maior probabilidade é também o par com a máxima probabilidade posterior:

In [35]:
pd.Series(posterior_pmf.max_prob(),
          index=['slope', 'inter'])

Assim, podemos confirmar que os resultados da actualização Bayesian são consistentes com a estimativa de máxima probabilidade calculada pela StatsModels:

In [36]:
results.params

São aproximadamente as mesmas, dentro da precisão da grelha que estamos a utilizar.

Se desempilharmos o `Pmf` posterior, podemos fazer um gráfico de contorno da distribuição posterior conjunta.

In [37]:
from utils import plot_contour

joint_posterior = posterior_pmf.unstack()
plot_contour(joint_posterior)
decorate(title='Joint posterior distribution')

As ovais na curva de nível estão alinhadas ao longo de uma diagonal, o que indica que existe alguma correlação entre a "inclinação" e o "inverno" na distribuição posterior.

Mas a correlação é fraca, o que é uma das razões pelas quais subtraímos a temperatura média de lançamento quando calculámos `x`; centrar os dados minimiza a correlação entre os parâmetros.

**Exercício:** Para ver porque é que isto importa, volte atrás e defina `offset=60` e execute novamente a análise.

A inclinação deve ser a mesma, mas a intercepção será diferente.  E se se traçar a distribuição conjunta, os contornos obtidos serão alongados, indicando uma correlação mais forte entre os parâmetros estimados.  

Em teoria, esta correlação não é um problema, mas na prática é.  Com dados não centrados, a distribuição posterior é mais difundida, pelo que é mais difícil de cobrir com a distribuição prévia conjunta.

Centrar os dados maximiza a precisão das estimativas; com dados não centrados, temos de fazer mais cálculos para obter a mesma precisão.

## Distribuições Marginais

Finalmente, podemos extrair as distribuições marginais.

In [38]:
from utils import marginal

marginal_inter = marginal(joint_posterior, 0)
marginal_slope = marginal(joint_posterior, 1)

Aqui está a distribuição posterior do `inter`.

In [39]:

marginal_inter.plot(label='intercept', color='C4')

decorate(xlabel='Intercept',
         ylabel='PDF',
         title='Posterior marginal distribution of intercept')

E aqui está a distribuição posterior do "declive".

In [40]:
marginal_slope.plot(label='slope', color='C2')

decorate(xlabel='Slope',
         ylabel='PDF',
         title='Posterior marginal distribution of slope')

Aqui estão os meios posteriores.

In [41]:
pd.Series([marginal_inter.mean(), marginal_slope.mean()],
          index=['inter', 'slope'])

Ambas as distribuições marginais são moderadamente enviesadas, pelo que os meios posteriores são um pouco diferentes das estimativas pontuais.

In [42]:
results.params

## Transforming Distributions

Vamos interpretar estes parâmetros.  Recordemos que a intercepção é a probabilidade de log da hipótese quando $x$ é 0, que é quando a temperatura é de cerca de 70 graus F (o valor de `offset`).

Assim, podemos interpretar as quantidades em `marginal_inter` como probabilidades de registo.

Para as converter em probabilidades, utilizarei a seguinte função, que transforma as quantidades num `Pmf` através da aplicação de uma dada função:

In [43]:
def transform(pmf, func):
    """Transform the quantities in a Pmf."""
    ps = pmf.ps
    qs = func(pmf.qs)
    return Pmf(ps, qs, copy=True)

Se chamarmos `transformação` e passarmos de `expit` como parâmetro, transforma as probabilidades de log em `marginal_inter` em probabilidades e devolve a distribuição posterior de `inter` expressa em termos de probabilidades.

In [44]:
marginal_probs = transform(marginal_inter, expit)

O `Pmf' fornece um método de `transformação' que faz a mesma coisa.

In [45]:
marginal_probs = marginal_inter.transform(expit)

Aqui está a distribuição posterior para a probabilidade de danos a 70 graus F.

In [46]:
marginal_probs.plot(color='C1')

decorate(xlabel='Probability of damage at 70 deg F',
         ylabel='PDF',
         title='Posterior marginal distribution of probabilities')

A média desta distribuição é de cerca de 22%, que é a probabilidade de danos a 70 graus F, de acordo com o modelo.

In [47]:
mean_prob = marginal_probs.mean()
mean_prob

Este resultado mostra a segunda razão pela qual defini `x` como sendo zero quando a temperatura é 70 graus F; desta forma, a intercepção corresponde à probabilidade de dano a uma temperatura relevante, em vez de 0 graus F.

Agora vamos olhar mais de perto para o declive estimado.  No modelo logístico, o parâmetro $\beta_1$ é o log do rácio de verosimilhança.  

Assim, podemos interpretar as quantidades em `marginal_slope` como log likelihood ratios, e podemos utilizar `exp` para as transformar em probabilidades (também conhecidas como factores Bayes).

In [48]:
marginal_lr = marginal_slope.transform(np.exp)

O resultado é a distribuição posterior dos rácios de probabilidade; aqui está o que parece.

In [49]:
marginal_lr.plot(color='C3')

decorate(xlabel='Likelihood ratio of 1 deg F',
         ylabel='PDF',
         title='Posterior marginal distribution of likelihood ratios')

In [50]:
mean_lr = marginal_lr.mean()
mean_lr

A média desta distribuição é de cerca de 0,75, o que significa que cada grau adicional de Fahrenheit fornece provas contra a possibilidade de danos, com um rácio de probabilidade (factor Bayes) de 0,75.

Aviso:

* Calculei a média posterior da probabilidade de dano a 70 graus F, transformando a distribuição marginal da intercepção para a distribuição marginal da probabilidade, e depois calculando a média.

* Calculei a média posterior do rácio de verosimilhança, transformando a distribuição marginal da inclinação para a distribuição marginal dos rácios de verosimilhança, e depois calculei a média.

Esta é a ordem correcta das operações, em oposição ao cálculo dos meios posteriores primeiro e depois a sua transformação.  

Para ver a diferença, vamos calcular os dois valores ao contrário. 

Aqui está a média posterior de `marginal_inter`, transformada numa probabilidade, em comparação com a média de `marginal_probs`.

In [51]:
expit(marginal_inter.mean()), marginal_probs.mean()

E aqui está a média posterior da "inclinação_marginal", transformada num rácio de probabilidade, em comparação com a média da "inclinação_marginal".

In [52]:
np.exp(marginal_slope.mean()), marginal_lr.mean()

Neste exemplo, as diferenças não são enormes, mas podem ser.

Como regra geral, transformar primeiro, depois calcular as estatísticas sumárias.

## Distribuições Preditivas

No modelo logístico, os parâmetros são interpretáveis, pelo menos após transformação.  Mas muitas vezes o que nos interessa são as previsões e não os parâmetros.  No problema do vaivém espacial, a previsão mais importante é: "Qual é a probabilidade de dano por O-ring se a temperatura exterior for de 31 graus F?

Para fazer essa previsão, vou retirar uma amostra de pares de parâmetros da distribuição posterior.

In [53]:
np.random.seed(17)

In [54]:
sample = posterior_pmf.choice(101)

O resultado é um conjunto de 101 tuplos, cada um representando um possível par de parâmetros.

Escolhi este tamanho de amostra para tornar o cálculo rápido.

Aumentar não mudaria muito os resultados, mas seriam um pouco mais precisos.

In [55]:
sample.shape

In [56]:
sample.dtype

In [57]:
type(sample[0])

Para gerar previsões, vou usar uma gama de temperaturas de 31 graus F (a temperatura quando o Challenger lançou) a 82 graus F (a temperatura mais alta observada).

In [58]:
temps = np.arange(31, 83)
xs = temps - offset

O laço seguinte utiliza `xs` e a amostra de parâmetros para construir um conjunto de probabilidades previstas.

In [59]:
pred = np.empty((len(sample), len(xs)))

for i, (slope, inter) in enumerate(sample):
    pred[i] = expit(inter + slope * xs)

O resultado tem uma coluna para cada valor em `x` e uma linha para cada elemento de `amostra`.  

Para ter uma rápida noção de como são as previsões, podemos percorrer as filas e traçá-las.

In [60]:
for ps in pred:
    plt.plot(temps, ps, color='C1', lw=0.5, alpha=0.4)
    
plot_data(data)

As linhas sobrepostas nesta figura dão uma sensação do valor mais provável a cada temperatura e do grau de incerteza.

Em cada coluna, calcularei a mediana para quantificar a tendência central e um intervalo credível de 90% para quantificar a incerteza.

O 'percentilnp.percentil' calcula os percentis dados; com o argumento 'eixo=0', calcula-os para cada coluna.

In [61]:
low, median, high = np.percentile(pred, [5, 50, 95], axis=0)

Os resultados são matrizes contendo probabilidades previstas para o limite inferior do IC 90%, a mediana, e o limite superior do IC.

Eis como eles são:

In [62]:
plt.fill_between(temps, low, high, color='C1', alpha=0.2)
plt.plot(temps, median, color='C1', label='logistic model')

plot_data(data)

De acordo com estes resultados, a probabilidade de danos nos anéis em O a 80 graus F está próxima dos 2%, mas há alguma incerteza quanto a essa previsão; o limite superior do IC é de cerca de 10%.

A 60 graus, a probabilidade de danos é perto de 80%, mas a IC é ainda mais ampla, de 48% a 97%.

Mas o objectivo principal do modelo é prever a probabilidade de danos a 31 graus F, e a resposta é pelo menos 97%, e a probabilidade de ser mais de 99,9%.

In [63]:
low = pd.Series(low, temps)
median = pd.Series(median, temps)
high = pd.Series(high, temps)

In [64]:
t = 80
print(median[t], (low[t], high[t]))

In [65]:
t = 60
print(median[t], (low[t], high[t]))

In [66]:
t = 31
print(median[t], (low[t], high[t]))

Uma conclusão que podemos tirar é a seguinte:  Se as pessoas responsáveis pelo lançamento do Challenger tivessem tido em conta todos os dados, e não apenas os sete incidentes de danos, poderiam ter previsto que a probabilidade de danos a 31 graus F era quase certa.  Se o tivessem feito, parece provável que tivessem adiado o lançamento.

Ao mesmo tempo, se considerassem a figura anterior, poderiam ter percebido que o modelo faz previsões que vão muito além dos dados.  Quando extrapolamos assim, temos de recordar não só a incerteza quantificada pelo modelo, que exprimimos como um intervalo credível; temos também de considerar a possibilidade de que o modelo em si não é fiável.

Este exemplo baseia-se num modelo logístico, que pressupõe que cada grau adicional de temperatura contribui com a mesma quantidade de provas a favor (ou contra) a possibilidade de danos.  Dentro de uma gama estreita de temperaturas, essa pode ser uma suposição razoável, especialmente se for apoiada por dados.  Mas numa gama mais ampla, e para além dos limites dos dados, a realidade não tem obrigação de se cingir ao modelo.

## Empirical Bayes

Neste capítulo utilizei StatsModels para calcular os parâmetros que maximizam a probabilidade dos dados, e depois utilizei essas estimativas para escolher os limites das distribuições prévias uniformes.

Pode ter-lhe ocorrido que este processo utiliza os dados duas vezes, uma para escolher os antecedentes e outra para fazer a actualização.  Se isso o incomoda, não está sozinho.

O processo que utilizei é um exemplo do que se chama [Empirical Bayes method](https://en.wikipedia.org/wiki/Empirical_Bayes_method), embora não me pareça que seja um nome particularmente bom para ele.

Embora possa parecer problemático utilizar os dados duas vezes, nestes exemplos, não o é.  Para ver porquê, considerar uma alternativa: em vez de utilizar os parâmetros estimados para escolher os limites da distribuição anterior, poderia ter utilizado distribuições uniformes com gamas muito mais amplas.

Nesse caso, os resultados seriam os mesmos; a única diferença é que eu passaria mais tempo a calcular probabilidades para parâmetros onde as probabilidades posteriores são negligenciavelmente pequenas.

Assim, pode pensar nesta versão do Empirical Bayes como uma optimização que minimiza a computação, colocando as distribuições anteriores onde a probabilidade dos dados vale a pena computar.

Esta optimização não afecta os resultados, pelo que não "conta duplamente" os dados.

## Resumo

Até agora, vimos três formas de representar graus de confiança numa hipótese: probabilidade, probabilidades, e probabilidades de registo.

Quando escrevemos a Regra de Bayes em termos de probabilidades logísticas, uma actualização Bayesiana é a soma do anterior e da probabilidade; neste sentido, as estatísticas Bayesianas são a aritmética das hipóteses e provas.

Esta forma do Teorema de Bayes é também a base da regressão logística, que usávamos para inferir parâmetros e fazer previsões.  No problema do vaivém espacial, modelámos a relação entre a temperatura e a probabilidade de danos, e mostrámos que a catástrofe do Challenger poderia ter sido previsível.  Mas este exemplo é também um aviso sobre os perigos da utilização de um modelo para extrapolar muito para além dos dados.

Nos exercícios abaixo terá a oportunidade de praticar o material deste capítulo, usando as probabilidades logísticas para avaliar um pundit político e usando a regressão logística para modelar as taxas de diagnóstico de Distúrbio de Défice de Atenção e Hiperactividade (DDAH).

No próximo capítulo passaremos da regressão logística para a regressão linear, que utilizaremos para modelar mudanças ao longo do tempo na temperatura, queda de neve, e o recorde mundial da maratona.

## Exercícios

**Exercício:** Suponha que um erudito político afirma ser capaz de prever o resultado das eleições, mas em vez de escolher um vencedor, eles dão a cada candidato uma probabilidade de ganhar.

Com esse tipo de previsão, pode ser difícil dizer se está certo ou errado. 

Por exemplo, suponhamos que o Pundit diz que Alice tem 70% de hipóteses de vencer Bob, e depois Bob ganha as eleições.  Será que isso significa que o erudito estava errado?

Uma forma de responder a esta pergunta é considerar duas hipóteses:

* `H`: O algoritmo do Pundit é legítimo; as probabilidades que produz são correctas no sentido em que reflectem com precisão as probabilidades de vitória dos candidatos.

* 'não H': O algoritmo do Pundit é falso; as probabilidades que produz são valores aleatórios com uma média de 50%.

Se o conhecedor diz que Alice tem 70% de hipóteses de ganhar, e ela tem, isso fornece provas a favor de `H` com rácio de probabilidade 70/50.

Se o conhecedor diz que Alice tem 70% de hipóteses de ganhar, e ela perde, isso é prova contra `H` com um rácio de probabilidade de 50/30.

Suponhamos que começamos com alguma confiança no algoritmo, pelo que as probabilidades anteriores são de 4 para 1.  E suponhamos que o Pundit gera previsões para três eleições:

* Nas primeiras eleições, o Pundit diz que Alice tem 70% de hipóteses de ganhar e ela tem.

* Nas segundas eleições, o Pundit diz que Bob tem 30% de probabilidades de ganhar e ele ganha.

* Nas terceiras eleições, o Pundit diz que Carol tem 90% de hipóteses de ganhar e ela tem.

Qual é o log likelihood ratio para cada um destes resultados?  Utilize a forma de log-odds da Regra de Bayes para calcular as probabilidades de log posterior para `H` após estes resultados.  No total, estes resultados aumentam ou diminuem a sua confiança no "pundit"?

Se estiver interessado neste tópico, pode [read more about it in this blog post](http://allendowney.blogspot.com/2016/11/why-are-we-so-surprised.html).

In [67]:
# Solution goes here

In [68]:
# Solution goes here

In [69]:
# Solution goes here

**Exercício:** Um artigo no *New England Journal of Medicine* relata resultados de um estudo que analisou a taxa de diagnóstico de Distúrbio de Défice de Atenção e Hiperactividade (DDAH) em função do mês de nascimento: ["Attention Deficitâ€“Hyperactivity Disorder and Month of School Enrollment"](https://www.nejm.org/doi/10.1056/NEJMoa1806828).

Verificaram que as crianças nascidas em Junho, Julho e Agosto tinham uma probabilidade substancialmente maior de serem diagnosticadas com TDAH, em comparação com as crianças nascidas em Setembro, mas apenas em estados que utilizam um corte de Setembro para as crianças entrarem no jardim-de-infância.  Nestes estados, as crianças nascidas em Agosto começam a escola quase um ano mais cedo do que as crianças nascidas em Setembro.  Os autores do estudo sugerem que a causa é "variação de comportamento baseada na idade que pode ser atribuída à TDAH, e não à idade mais jovem das crianças". 

Utilizar os métodos deste capítulo para estimar a probabilidade de diagnóstico em função do mês de nascimento.

O caderno para este capítulo fornece os dados e algumas sugestões para começar.

O artigo inclui esta figura:

<img width="500" src="https://www.nejm.org/na101/home/literatum/publisher/mms/journals/content/nejm/2018/nejm_2018.379.issue-22/nejmoa1806828/20190131/images/img_xlarge/nejmoa1806828_f1.jpeg">

Na minha opinião, esta representação dos dados não mostra o efeito tão claramente quanto poderia.

Mas a figura inclui os dados em bruto, para que possamos analisá-los nós próprios.

Nota: há um erro na figura, confirmado por correspondência pessoal:

> Maio e Junho [diagnoses] are reversed. May should be 317 (not 287) e Junho devem ser 287 (e não 317).

Portanto, aqui estão os dados corrigidos, onde `n` é o número de crianças nascidas em cada mês, a partir de Janeiro, e `k' é o número de crianças diagnosticadas com TDAH.

In [70]:
n = np.array([32690, 31238, 34405, 34565, 34977, 34415, 
                   36577, 36319, 35353, 34405, 31285, 31617])

k = np.array([265, 280, 307, 312, 317, 287, 
                      320, 309, 225, 240, 232, 243])

Primeiro, vou "enrolar" os dados para que comecem em Setembro em vez de Janeiro.

In [71]:
x = np.arange(12)
n = np.roll(n, -8)
k = np.roll(k, -8)

E vou colocá-lo num "DataFrame" com uma fila para cada mês e a taxa de diagnóstico por 10.000.

In [72]:
adhd = pd.DataFrame(dict(x=x, k=k, n=n))
adhd['rate'] = adhd['k'] / adhd['n'] * 10000
adhd

Eis como são as taxas de diagnóstico.

In [73]:
def plot_adhd(adhd):
    plt.plot(adhd['x'], adhd['rate'], 'o', 
             label='data', color='C0', alpha=0.4)
    
    plt.axvline(5.5, color='gray', alpha=0.2)
    plt.text(6, 64, 'Younger than average')
    plt.text(5, 64, 'Older than average', horizontalalignment='right')

    decorate(xlabel='Birth date, months after cutoff',
             ylabel='Diagnosis rate per 10,000')

In [74]:
plot_adhd(adhd)

Durante os primeiros 9 meses, de Setembro a Maio, vemos o que esperaríamos se alguns dos diagnósticos em excesso fossem devidos a "variação de comportamento baseada na idade".  Para cada mês de diferença de idade, vemos um aumento no número de diagnósticos.

Este padrão decompõe-se durante os últimos três meses, Junho, Julho e Agosto.  Isto pode ser explicado por variações aleatórias, mas também pode ser devido à manipulação dos pais; se alguns pais retiverem as crianças nascidas perto do prazo, as observações para este mês incluiriam uma mistura de crianças relativamente velhas para a sua classificação e, portanto, menos susceptíveis de serem diagnosticadas.

Infelizmente, o conjunto de dados inclui apenas o mês de nascimento, não o ano, pelo que não sabemos as idades reais destes estudantes quando começaram a escola.  Contudo, podemos utilizar os primeiros nove meses para estimar o efeito da idade na taxa de diagnóstico; depois podemos pensar sobre o que fazer com os outros três meses.

Utilizar os métodos deste capítulo para estimar a probabilidade de diagnóstico em função do mês de nascimento.

Comece com as seguintes distribuições prévias.

In [75]:
qs = np.linspace(-5.2, -4.6, num=51)
prior_inter = make_uniform(qs, 'Intercept')

In [76]:
qs = np.linspace(0.0, 0.08, num=51)
prior_slope = make_uniform(qs, 'Slope')

1. Fazer uma distribuição prévia conjunta e actualizá-la utilizando os dados para os primeiros nove meses.

2. Em seguida, retirar uma amostra da distribuição posterior e utilizá-la para calcular a probabilidade mediana de diagnóstico para cada mês e um intervalo credível de 90%.

3. Como exercício bónus, fazer uma segunda actualização utilizando os dados dos últimos três meses, mas tratando o número de diagnósticos observados como um limite inferior ao número de diagnósticos que haveria se nenhuma criança fosse retida.

In [77]:
# Solution goes here

In [78]:
# Solution goes here

In [79]:
# Solution goes here

In [80]:
# Solution goes here

In [81]:
# Solution goes here

In [82]:
# Solution goes here

In [83]:
# Solution goes here

In [84]:
# Solution goes here

In [85]:
# Solution goes here

In [86]:
# Solution goes here

In [87]:
# Solution goes here

In [88]:
# Solution goes here

In [89]:
# Solution goes here

In [90]:
# Solution goes here

In [91]:
# Solution goes here

In [92]:
# Solution goes here

In [93]:
# Solution goes here